In [8]:
import os
from dotenv import load_dotenv
from langchain_groq import ChatGroq

load_dotenv()
model = ChatGroq(model="llama-3.3-70b-versatile")

In [9]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful assistant. Answer in {language}."),
    MessagesPlaceholder(variable_name="messages"),
])

chain = prompt | model  # <-- QUAN TRỌNG: tạo chain

In [10]:
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory

store = {}
def get_session_history(session_id: str):
    if session_id not in store:
        store[session_id] = ChatMessageHistory()
    return store[session_id]

# BỌC CHAIN, KHÔNG BỌC MODEL
with_message_history = RunnableWithMessageHistory(
    chain,  # <-- PHẢI LÀ CHAIN, KHÔNG PHẢI MODEL
    get_session_history,
    input_messages_key="messages",
)

In [11]:
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory

store = {}
def get_session_history(session_id: str)->BaseChatMessageHistory:
    if session_id not in store:
        store[session_id] = ChatMessageHistory()
    return store[session_id]

with_message_history = RunnableWithMessageHistory(
    model,
    get_session_history,
    session_id="chat1"
)

In [12]:
config = {"configurable":{"session_id":"chat1"}}

In [14]:
from langchain_core.messages import HumanMessage
with_message_history.invoke(
  [HumanMessage(content="Hi, My name is Giang and I am a student")],
  config=config
)

AIMessage(content="Hello Giang, nice to meet you. Welcome to our conversation. What are you studying, if you don't mind me asking?", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 27, 'prompt_tokens': 46, 'total_tokens': 73, 'completion_time': 0.083798121, 'completion_tokens_details': None, 'prompt_time': 0.002199611, 'prompt_tokens_details': None, 'queue_time': 0.058151839, 'total_time': 0.085997732}, 'model_name': 'llama-3.3-70b-versatile', 'system_fingerprint': 'fp_68f543a7cc', 'service_tier': 'on_demand', 'finish_reason': 'stop', 'logprobs': None, 'model_provider': 'groq'}, id='lc_run--019bbac0-c092-7a90-948f-3b7706a0cfc1-0', tool_calls=[], invalid_tool_calls=[], usage_metadata={'input_tokens': 46, 'output_tokens': 27, 'total_tokens': 73})

In [15]:
config1 = {"configurable":{"session_id":"chat1"}}
response = with_message_history.invoke(
  [HumanMessage(content="Hey my name is Giang")],
  config=config1
)
response.content

"Hello again Giang. We've already met, but it's nice to see you again. Is there something I can help you with or would you like to chat?"

In [16]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.messages import HumanMessage

# 1. Định nghĩa lại Prompt hỗ trợ input text
prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful assistant. Answer all the question to the best of your ability."),
    MessagesPlaceholder(variable_name="messages"),
    # ("human", "{input}")                          
])

chain = prompt | model

chain.invoke({
    # "input": "Hi my name is Giang",
    "messages": [HumanMessage(content="Hi my name is Giang")]
})




AIMessage(content="Hello Giang, it's nice to meet you. Is there something I can help you with or would you like to chat?", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 26, 'prompt_tokens': 57, 'total_tokens': 83, 'completion_time': 0.060933432, 'completion_tokens_details': None, 'prompt_time': 0.010174603, 'prompt_tokens_details': None, 'queue_time': 0.05821872, 'total_time': 0.071108035}, 'model_name': 'llama-3.3-70b-versatile', 'system_fingerprint': 'fp_c06d5113ec', 'service_tier': 'on_demand', 'finish_reason': 'stop', 'logprobs': None, 'model_provider': 'groq'}, id='lc_run--019bbac0-cc4d-78a1-9334-cdf8da205eca-0', tool_calls=[], invalid_tool_calls=[], usage_metadata={'input_tokens': 57, 'output_tokens': 26, 'total_tokens': 83})

In [17]:
with_message_history  =  RunnableWithMessageHistory(chain,get_session_history)

In [18]:
config = {
    "configurable": {
        "session_id": "chat3"
    }
}

response = with_message_history.invoke(
    [HumanMessage(content="Hi my name is Giang")],
   config=config)

response


AIMessage(content="Hello Giang, it's nice to meet you. Is there something I can help you with or would you like to chat?", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 26, 'prompt_tokens': 57, 'total_tokens': 83, 'completion_time': 0.046405198, 'completion_tokens_details': None, 'prompt_time': 0.001904345, 'prompt_tokens_details': None, 'queue_time': 0.059224335, 'total_time': 0.048309543}, 'model_name': 'llama-3.3-70b-versatile', 'system_fingerprint': 'fp_68f543a7cc', 'service_tier': 'on_demand', 'finish_reason': 'stop', 'logprobs': None, 'model_provider': 'groq'}, id='lc_run--019bbac0-dd80-7522-95a5-cc9b55c4fa84-0', tool_calls=[], invalid_tool_calls=[], usage_metadata={'input_tokens': 57, 'output_tokens': 26, 'total_tokens': 83})

In [19]:
prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful assistant. Answer all question to the best of your ability in {language}"),
    MessagesPlaceholder(variable_name="messages")
])

chain = prompt | model



In [20]:
response = chain.invoke({"messages":[HumanMessage(content="Hello, my name is Giang")],"language":"Vietnamese"}) 
response

AIMessage(content='Xin chào Giang! Rất vui được gặp bạn. Tôi có thể giúp gì cho bạn hôm nay?', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 24, 'prompt_tokens': 58, 'total_tokens': 82, 'completion_time': 0.062201793, 'completion_tokens_details': None, 'prompt_time': 0.005212, 'prompt_tokens_details': None, 'queue_time': 0.05854859, 'total_time': 0.067413793}, 'model_name': 'llama-3.3-70b-versatile', 'system_fingerprint': 'fp_c06d5113ec', 'service_tier': 'on_demand', 'finish_reason': 'stop', 'logprobs': None, 'model_provider': 'groq'}, id='lc_run--019bbac0-ed57-7932-897c-ad3e45984839-0', tool_calls=[], invalid_tool_calls=[], usage_metadata={'input_tokens': 58, 'output_tokens': 24, 'total_tokens': 82})

In [21]:
with_message_history= RunnableWithMessageHistory(
  chain,
  get_session_history,
  input_message_key="message"
)

In [22]:
from langchain_core.messages import HumanMessage,AIMessage

config = {"configurable": {"session_id": "chat_test"}}
response = with_message_history.invoke(
    {
        "messages": [HumanMessage(content="What is my name?")],
        "language": "Vietnamese"
    },
    config=config
)
print(response.content)

KeyError: 'input'

In [28]:
from langchain_core.messages import SystemMessage, trim_messages
from langchain_core.messages import HumanMessage,AIMessage

trimmer = trim_messages(
  max_tokens=45,
  strategy="last",
  token_counter=model,
  include_system=True,
  allow_partial=False,
  start_on = "human"
)

message = [
  SystemMessage(content="You are a helpful assistant."),
  HumanMessage(content="Hello, how are you?"),
  AIMessage(content="hi!"),
  HumanMessage(content="I like vanilla ice cream."),
  AIMessage(content="nice"),
  HumanMessage(content="whats 2+2"),
  AIMessage(content="4"),
  HumanMessage(content="thanks"),
  AIMessage(content="no problem"),
  HumanMessage(content="Having fun"),
  AIMessage(content="yes!")
]
trimmer.invoke(message)


[SystemMessage(content='You are a helpful assistant.', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='I like vanilla ice cream.', additional_kwargs={}, response_metadata={}),
 AIMessage(content='nice', additional_kwargs={}, response_metadata={}, tool_calls=[], invalid_tool_calls=[]),
 HumanMessage(content='whats 2+2', additional_kwargs={}, response_metadata={}),
 AIMessage(content='4', additional_kwargs={}, response_metadata={}, tool_calls=[], invalid_tool_calls=[]),
 HumanMessage(content='thanks', additional_kwargs={}, response_metadata={}),
 AIMessage(content='no problem', additional_kwargs={}, response_metadata={}, tool_calls=[], invalid_tool_calls=[]),
 HumanMessage(content='Having fun', additional_kwargs={}, response_metadata={}),
 AIMessage(content='yes!', additional_kwargs={}, response_metadata={}, tool_calls=[], invalid_tool_calls=[])]

In [30]:
from operator import itemgetter

from langchain_core.runnables import RunnablePassthrough
chain=(
  RunnablePassthrough.assign(messages=itemgetter("messages")| trimmer)
  |prompt
  |model
)
chain.invoke(
  {
  "messages":message + [HumanMessage(content="what match problem did i ask")],
  "language":"Vietnamese"}
)

AIMessage(content='Bạn đã hỏi tôi giải một bài toán đơn giản là 2 + 2.', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 18, 'prompt_tokens': 109, 'total_tokens': 127, 'completion_time': 0.063653478, 'completion_tokens_details': None, 'prompt_time': 0.004639124, 'prompt_tokens_details': None, 'queue_time': 0.058089516, 'total_time': 0.068292602}, 'model_name': 'llama-3.3-70b-versatile', 'system_fingerprint': 'fp_dae98b5ecb', 'service_tier': 'on_demand', 'finish_reason': 'stop', 'logprobs': None, 'model_provider': 'groq'}, id='lc_run--019bbac3-9cec-7171-a6b2-13776a316b78-0', tool_calls=[], invalid_tool_calls=[], usage_metadata={'input_tokens': 109, 'output_tokens': 18, 'total_tokens': 127})

In [32]:
with_message_history = RunnableWithMessageHistory(
  chain,
  get_session_history,
  input_messages_key="messages"
)

config = {"configurable":{"session_id":"chat6"}}



In [ ]:

chain.invoke(
  {
  "messages":message + [HumanMessage(content="what is my name")],
  "language":"Vietnamese"},
  config=config
)

AIMessage(content='Tôi không biết tên của bạn. Chúng ta vừa bắt đầu trò chuyện, và tôi chưa được bạn chia sẻ thông tin về tên của mình. Bạn có muốn chia sẻ tên của mình không?', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 40, 'prompt_tokens': 107, 'total_tokens': 147, 'completion_time': 0.182374079, 'completion_tokens_details': None, 'prompt_time': 0.005432769, 'prompt_tokens_details': None, 'queue_time': 0.053315811, 'total_time': 0.187806848}, 'model_name': 'llama-3.3-70b-versatile', 'system_fingerprint': 'fp_c06d5113ec', 'service_tier': 'on_demand', 'finish_reason': 'stop', 'logprobs': None, 'model_provider': 'groq'}, id='lc_run--019bbac7-1695-7800-9a06-32089e842a04-0', tool_calls=[], invalid_tool_calls=[], usage_metadata={'input_tokens': 107, 'output_tokens': 40, 'total_tokens': 147})

: 